A few key concepts that we will revisit in this walk-through are:

1. Apply String Indexer method to find the index of the categorical columns

2. Apply OneHot encoding for the categorical columns

3. Apply String indexer for the output variable “label” column

4. VectorAssembler is applied for both categorical columns and numeric columns. VectorAssembler is a transformer that combines a given list of columns into a single vector column.

**Step 1**: Install Spark

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [ ]:
# install findspark using pip
!pip install -q findspark

In [ ]:
!pip3 install pyspark==3.0.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.8/204.8 MB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 KB 19.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.0.2-py2.py3-none-any.whl size=205186690 sha256=7f573739bfb5231dea41e2aceb3ce2062f51d7d4fc80cd62d22f4ca3a5ec54a2
  Stored in directory: /root/.cache/pip/wheels/aa/8e/b9/ed8017fb2997a648f5868a4b728881f320e3d1bd2b0274f137
Successfully built pyspark


In [ ]:
import findspark
findspark.init()

In [ ]:
# Import SparkSession
from pyspark.sql import SparkSession
# Create a Spark Session
spark = SparkSession.builder.master("local[*]").getOrCreate()
# Check Spark Session Information
spark


In [ ]:
# Create spark_session
spark_session = SparkSession.builder.getOrCreate()

In [ ]:
sc = spark_session.sparkContext

# The objective of this exercise is to predict that prices of houses in California

**Step 6**: Read in the data file

In [ ]:
# Load in the data
rdd = sc.textFile('/content/drive/My Drive/Colab Notebooks/datasets/cal_housing.data.csv')

**Step 7**: Read in the headers for the data file read in above

In [ ]:
# Load in the header
header = sc.textFile('/content/drive/My Drive/Colab Notebooks/datasets/cal_housing.domain')

**Step 8**: Output the first 2 rows of the data set which was read in Step 7

In [ ]:
rdd.take(2)

['-122.230000,37.880000,41.000000,880.000000,129.000000,322.000000,126.000000,8.325200,452600.000000',
 '-122.220000,37.860000,21.000000,7099.000000,1106.000000,2401.000000,1138.000000,8.301400,358500.000000']

**Step 9**:  Split the 2 lines on comas and examine the first 2 lines

In [ ]:
# Split the lines on commas
rdd = rdd.map(lambda line: line.split(","))

# Examine the first 2 lines 
rdd.take(2)

[['-122.230000',
  '37.880000',
  '41.000000',
  '880.000000',
  '129.000000',
  '322.000000',
  '126.000000',
  '8.325200',
  '452600.000000'],
 ['-122.220000',
  '37.860000',
  '21.000000',
  '7099.000000',
  '1106.000000',
  '2401.000000',
  '1138.000000',
  '8.301400',
  '358500.000000']]

In [ ]:
# Import the necessary modules 
from pyspark.sql import Row

**Step 10**: Convert the RDD into a DataFrame - it is easier to work with a DataFrame

In [ ]:
# Map the RDD to a DF
df = rdd.map(lambda line: Row(longitude=line[0], 
                              latitude=line[1], 
                              housingMedianAge=line[2],
                              totalRooms=line[3],
                              totalBedRooms=line[4],
                              population=line[5], 
                              households=line[6],
                              medianIncome=line[7],
                              medianHouseValue=line[8])).toDF()

**Step 11**: Display the contents of the DataFrame

In [ ]:
df.show()

+-----------+---------+----------------+-----------+-------------+-----------+-----------+------------+----------------+
|  longitude| latitude|housingMedianAge| totalRooms|totalBedRooms| population| households|medianIncome|medianHouseValue|
+-----------+---------+----------------+-----------+-------------+-----------+-----------+------------+----------------+
|-122.230000|37.880000|       41.000000| 880.000000|   129.000000| 322.000000| 126.000000|    8.325200|   452600.000000|
|-122.220000|37.860000|       21.000000|7099.000000|  1106.000000|2401.000000|1138.000000|    8.301400|   358500.000000|
|-122.240000|37.850000|       52.000000|1467.000000|   190.000000| 496.000000| 177.000000|    7.257400|   352100.000000|
|-122.250000|37.850000|       52.000000|1274.000000|   235.000000| 558.000000| 219.000000|    5.643100|   341300.000000|
|-122.250000|37.850000|       52.000000|1627.000000|   280.000000| 565.000000| 259.000000|    3.846200|   342200.000000|
|-122.250000|37.850000|       52

**Step 12**: Display the data types

In [ ]:
df.dtypes

[('longitude', 'string'),
 ('latitude', 'string'),
 ('housingMedianAge', 'string'),
 ('totalRooms', 'string'),
 ('totalBedRooms', 'string'),
 ('population', 'string'),
 ('households', 'string'),
 ('medianIncome', 'string'),
 ('medianHouseValue', 'string')]

In [ ]:
# Import all from `sql.types`
from pyspark.sql.types import *


**Step 13**: Function that converts the data types of the DataFrame columns

In [ ]:
# Write a custom function to convert the data type of DataFrame columns
def convertColumn(df, names, newType):
  for name in names: 
     df = df.withColumn(name, df[name].cast(newType))
  return df 

In [ ]:
# Assign all column names to `columns`
columns = ['households', 'housingMedianAge', 'latitude', 'longitude', 'medianHouseValue', 'medianIncome', 'population', 'totalBedRooms', 'totalRooms']

**Step 14**: Convert the data types of the above mentioned columns into a float type

In [ ]:
from pyspark.sql.types import *
# Conver the `df` columns to `FloatType()`
df = convertColumn(df, columns, FloatType())

**Step 15**: Confirm that the data type has been converted into float

In [ ]:
df.dtypes

[('longitude', 'float'),
 ('latitude', 'float'),
 ('housingMedianAge', 'float'),
 ('totalRooms', 'float'),
 ('totalBedRooms', 'float'),
 ('population', 'float'),
 ('households', 'float'),
 ('medianIncome', 'float'),
 ('medianHouseValue', 'float')]

In [ ]:
# Print the schema of `df`
df.printSchema()

root
 |-- longitude: float (nullable = true)
 |-- latitude: float (nullable = true)
 |-- housingMedianAge: float (nullable = true)
 |-- totalRooms: float (nullable = true)
 |-- totalBedRooms: float (nullable = true)
 |-- population: float (nullable = true)
 |-- households: float (nullable = true)
 |-- medianIncome: float (nullable = true)
 |-- medianHouseValue: float (nullable = true)



In [ ]:
df.describe().show()

+-------+-------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+
|summary|          longitude|          latitude|  housingMedianAge|        totalRooms|    totalBedRooms|        population|        households|      medianIncome|  medianHouseValue|
+-------+-------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+
|  count|              20640|             20640|             20640|             20640|            20640|             20640|             20640|             20640|             20640|
|   mean|-119.56970444871473| 35.63186143109965|28.639486434108527|2635.7630813953488|537.8980135658915|1425.4767441860465| 499.5396802325581|3.8706710030346416|206855.81690891474|
| stddev| 2.0035317429328914|2.1359523806029554|12.585557612111613|2181.6152515827994|421.24790


You should probably standardize your data, as you have seen that the range of minimum and maximum values is quite large.

Your dependent variable is also quite large; you should adjust the values slightly.

**Step 16**: Processing the data

In [ ]:
# Import all from `sql.functions` 
from pyspark.sql.functions import *

# Adjust the values of `medianHouseValue` - surface the house values in units of 100,000
df = df.withColumn("medianHouseValue", col("medianHouseValue")/100000)

# Show the first 2 lines of `df`
df.take(2)

[Row(longitude=-122.2300033569336, latitude=37.880001068115234, housingMedianAge=41.0, totalRooms=880.0, totalBedRooms=129.0, population=322.0, households=126.0, medianIncome=8.325200080871582, medianHouseValue=4.526),
 Row(longitude=-122.22000122070312, latitude=37.86000061035156, housingMedianAge=21.0, totalRooms=7099.0, totalBedRooms=1106.0, population=2401.0, households=1138.0, medianIncome=8.301400184631348, medianHouseValue=3.585)]

In [ ]:
from pyspark.sql.functions import *

# Divide `totalRooms` by `households`
roomsPerHousehold = df.select(col("totalRooms")/col("households"))

# Divide `population` by `households`
populationPerHousehold = df.select(col("population")/col("households"))

# Divide `totalBedRooms` by `totalRooms`
bedroomsPerRoom = df.select(col("totalBedRooms")/col("totalRooms"))

# Add the new columns to `df`
df = df.withColumn("roomsPerHousehold", col("totalRooms")/col("households")) \
   .withColumn("populationPerHousehold", col("population")/col("households")) \
   .withColumn("bedroomsPerRoom", col("totalBedRooms")/col("totalRooms"))
   
# Inspect the result
df.first()

Row(longitude=-122.2300033569336, latitude=37.880001068115234, housingMedianAge=41.0, totalRooms=880.0, totalBedRooms=129.0, population=322.0, households=126.0, medianIncome=8.325200080871582, medianHouseValue=4.526, roomsPerHousehold=6.984126984126984, populationPerHousehold=2.5555555555555554, bedroomsPerRoom=0.14659090909090908)

In [ ]:
# Re-order and select columns
df = df.select("medianHouseValue", 
              "totalBedRooms", 
              "population", 
              "households", 
              "medianIncome", 
              "roomsPerHousehold", 
              "populationPerHousehold", 
              "bedroomsPerRoom")

In [ ]:
df.show(10)

+----------------+-------------+----------+----------+------------+------------------+----------------------+-------------------+
|medianHouseValue|totalBedRooms|population|households|medianIncome| roomsPerHousehold|populationPerHousehold|    bedroomsPerRoom|
+----------------+-------------+----------+----------+------------+------------------+----------------------+-------------------+
|           4.526|        129.0|     322.0|     126.0|      8.3252| 6.984126984126984|    2.5555555555555554|0.14659090909090908|
|           3.585|       1106.0|    2401.0|    1138.0|      8.3014| 6.238137082601054|     2.109841827768014|0.15579659106916466|
|           3.521|        190.0|     496.0|     177.0|      7.2574| 8.288135593220339|    2.8022598870056497|0.12951601908657123|
|           3.413|        235.0|     558.0|     219.0|      5.6431|5.8173515981735155|     2.547945205479452|0.18445839874411302|
|           3.422|        280.0|     565.0|     259.0|      3.8462| 6.281853281853282|    

**Step 17**: Specifying the label and the features - the label in this case is the dependent variable i.e. **medianHouseValue**

In [ ]:
# Import `DenseVector`
# A Dense Vector is used to store arrays of values for use in PySpark.
from pyspark.ml.linalg import DenseVector

# Define the `input_data` 
input_data = df.rdd.map(lambda x: (x[0], DenseVector(x[1:])))

# Replace `df` with the new DataFrame
df = spark_session.createDataFrame(input_data, ["label", "features"])

label = df.rdd.map(lambda x: x.label)
features = df.rdd.map(lambda x: x.features)

In [ ]:
print(input_data)

PythonRDD[50] at RDD at PythonRDD.scala:53


**Step 18**: Scaling the features using 'StandardScaler' - standardizes a feature of the model by subtracting the mean and then scaling to unit variance. Unit variance means dividing all the values by the standard deviation.

**Additional Information**: https://towardsdatascience.com/scale-standardize-or-normalize-with-scikit-learn-6ccc7d176a02

In [ ]:
# Import `StandardScaler` 
from pyspark.ml.feature import StandardScaler

# Initialize the `standardScaler`
standardScaler = StandardScaler(inputCol="features", outputCol="features_scaled")

# Fit the DataFrame to the scaler
scaler = standardScaler.fit(df.select('features'))

# Transform the data in `df` with the scaler
scaled_df = scaler.transform(df)

# Inspect the result
scaled_df.take(2)

[Row(label=4.526, features=DenseVector([129.0, 322.0, 126.0, 8.3252, 6.9841, 2.5556, 0.1466]), features_scaled=DenseVector([0.3062, 0.2843, 0.3296, 4.3821, 2.8228, 0.2461, 2.5264])),
 Row(label=3.585, features=DenseVector([1106.0, 2401.0, 1138.0, 8.3014, 6.2381, 2.1098, 0.1558]), features_scaled=DenseVector([2.6255, 2.1202, 2.9765, 4.3696, 2.5213, 0.2031, 2.6851]))]

**Step 19**: Create the "Train/Test" split

In [ ]:
# Split the data into train and test sets
train_data, test_data = scaled_df.randomSplit([.8,.2],seed=1234)

In [ ]:
# Import `LinearRegression`
from pyspark.ml.regression import LinearRegression

# Initialize `lr`
lr = LinearRegression(labelCol="label", maxIter=100, regParam=0.3, elasticNetParam=0.8)


In [ ]:
train_data.show()

+-------+--------------------+--------------------+
|  label|            features|     features_scaled|
+-------+--------------------+--------------------+
|0.14999|[73.0,85.0,38.0,1...|[0.17329463000311...|
|0.14999|[267.0,628.0,225....|[0.63383104398397...|
|  0.175|[168.0,259.0,138....|[0.39881503891126...|
|   0.25|[33.0,64.0,27.0,0...|[0.07833866835757...|
|  0.266|[309.0,808.0,294....|[0.73353480371179...|
|  0.269|[543.0,1423.0,482...|[1.28902717933820...|
|    0.3|[183.0,500.0,177....|[0.43442352452834...|
|    0.3|[448.0,338.0,182....|[1.06350677043004...|
|  0.325|[49.0,51.0,20.0,4...|[0.11632105301578...|
|  0.329|[386.0,436.0,213....|[0.91632502987946...|
|  0.332|[131.0,511.0,124....|[0.31098077438914...|
|  0.344|[121.0,530.0,115....|[0.28724178397775...|
|  0.346|[208.0,660.0,188....|[0.49377100055680...|
|  0.366|[199.0,567.0,204....|[0.47240590918656...|
|  0.367|[238.0,425.0,157....|[0.56498797179096...|
|  0.375|[20.0,39.0,18.0,3...|[0.04747798082276...|
|  0.375|[19

In [ ]:
# Fit the data to the model
linearModel = lr.fit(train_data)

**Step 20**: Make the predictions

In [ ]:
# Make predictions on test data
predicted = linearModel.transform(test_data)


In [ ]:
# Retrieve the predictions and the "known" labels
predictions = predicted.select("prediction").rdd.map(lambda x: x[0])
labels = predicted.select("label").rdd.map(lambda x: x[0])


In [ ]:
# Combine the predictions and the label
predictionAndLabel = predictions.zip(labels).collect()

**Step 21**: Output the predictions and the associated labels

In [ ]:
# Print out first 5 instances of `predictionAndLabel` - Please note that the medianHouseValue was divided by 100000 in Step 17
predictionAndLabel[:15]

[(1.5755376157988699, 0.14999),
 (1.7341249044713924, 0.225),
 (1.5318778705905935, 0.388),
 (1.362865204899986, 0.394),
 (1.286302045413713, 0.396),
 (1.5233279429128062, 0.398),
 (1.4606101380914116, 0.4),
 (1.3705877149188916, 0.431),
 (1.3286655270672303, 0.44),
 (1.35619075944089, 0.441),
 (1.2721532889306606, 0.45),
 (1.3940310618213674, 0.455),
 (1.3423454047760115, 0.455),
 (1.4891558500524658, 0.455),
 (1.5159364196844893, 0.463)]

**Step 22**: Evaluating the model

**RMSE**: RMSE measures the differences between predicted values by the model and the actual values.The smaller the RMSE value is, the closer predicted and actual values are.

In [ ]:
linearModel.summary.rootMeanSquaredError

0.8757283763853735

**R-Squared** known as "Co-efficient of determination" illustrates the extent of the variability in the "MedianHouseValue" that can be explained by the Linear Regression model. The higher the R-squared, the better the model fits the underlying data.

In [ ]:
linearModel.summary.r2

0.42060409588578673

Only 42% of the variability in the "MedianHouseValues" is explained by the Linear Regression model.There is definitely room for improvement. You can play around with the parameters that you passed to your model.